In [ ]:
import duckdb
import pandas
import json

In [ ]:
conn = duckdb.connect(database=":memory:")

In [ ]:
query = f"SELECT * FROM 'all_properties.parquet'"

In [ ]:
df = conn.execute(query).fetchdf()

In [ ]:
df.info()

In [ ]:
json_field_values = json.loads(df.loc[1, 'field_values'])

In [ ]:
for i in range(len(json_field_values)):
    if 'provided' in json_field_values[i]['key']:
        print(json_field_values[i])